In [1]:
import pandas as pd
from multiprocessing import Pool, cpu_count
import numpy as np
from src.input.write_files import write_circuit_file
from src.input.read_files import find_files, numbers_from_file_name, normal_sec_size_int_from_file_name, number_from_text
from datetime import datetime
from src.algorithm.circuit_synthesis_vanilla import synthesise_circuit
from src.algorithm.execution import load_circuit_and_benchmark
import csv

print(f'There are {cpu_count()} threads available on this machine.')

There are 12 threads available on this machine.


In [2]:
sample_size = 2
lower_size = 1958
upper_size = 2048
step_size = 15

sizes = np.arange(lower_size, upper_size + step_size, step_size)
samples = np.arange(1, sample_size + 1, 1)

input_gen_processors = 10

input_dir = r'resources\section_size_comparison\input_matrices'
input_file_prefix = 'SingMat'

benchmark_processors = 5

output_dir = r'resources\section_size_comparison'
output_file_prefix = 'SectionSizeComparison'

output_file_fieldnames = ['size', 'run', 'sec_size', 'num_gates', 'process_time', 'initial_rss', 'final_rss']

In [3]:
input_gen_argument_tuples = ([(size, size * size, rf'..\{input_dir}\{input_file_prefix}_{size}_{sample}.txt') for size in sizes for sample in samples])

[print(argument_tuple) for argument_tuple in input_gen_argument_tuples[:min(10, len(input_gen_argument_tuples))]]

(1958, 3833764, '..\\resources\\section_size_comparison\\input_matrices\\SingMat_1958_1.txt')
(1958, 3833764, '..\\resources\\section_size_comparison\\input_matrices\\SingMat_1958_2.txt')
(1973, 3892729, '..\\resources\\section_size_comparison\\input_matrices\\SingMat_1973_1.txt')
(1973, 3892729, '..\\resources\\section_size_comparison\\input_matrices\\SingMat_1973_2.txt')
(1988, 3952144, '..\\resources\\section_size_comparison\\input_matrices\\SingMat_1988_1.txt')
(1988, 3952144, '..\\resources\\section_size_comparison\\input_matrices\\SingMat_1988_2.txt')
(2003, 4012009, '..\\resources\\section_size_comparison\\input_matrices\\SingMat_2003_1.txt')
(2003, 4012009, '..\\resources\\section_size_comparison\\input_matrices\\SingMat_2003_2.txt')
(2018, 4072324, '..\\resources\\section_size_comparison\\input_matrices\\SingMat_2018_1.txt')
(2018, 4072324, '..\\resources\\section_size_comparison\\input_matrices\\SingMat_2018_2.txt')


[None, None, None, None, None, None, None, None, None, None]

In [4]:
print(f'Generating {len(input_gen_argument_tuples)} circuits in size range from {lower_size} to {upper_size} ...')

if __name__ ==  '__main__':
    with Pool(processes = input_gen_processors) as pool:
        pool.starmap(write_circuit_file, input_gen_argument_tuples)

print(f'Saved circuits to {input_dir} ...')

Generating 14 circuits in size range from 1958 to 2048 ...
Saved circuits to resources\section_size_comparison\input_matrices ...


In [4]:
file_regex = rf'({input_file_prefix}).*\.txt'
size_regex = r'_\d+_'
sample_regex = r'_\d+\.'

input_file_names = find_files(rf'..\{input_dir}', file_regex)
input_file_names.sort(key = lambda input_file_name: numbers_from_file_name(input_file_name, size_regex, sample_regex))

[print(input_file_name) for input_file_name in input_file_names[:min(10, len(input_file_names))]]

SingMat_1958_1.txt
SingMat_1958_2.txt
SingMat_1973_1.txt
SingMat_1973_2.txt
SingMat_1988_1.txt
SingMat_1988_2.txt
SingMat_2003_1.txt
SingMat_2003_2.txt
SingMat_2018_1.txt
SingMat_2018_2.txt


[None, None, None, None, None, None, None, None, None, None]

In [5]:
section_sizes_df = pd.read_csv(rf'..\resources\optimization\SecSizes_2_5000.csv')
section_sizes_df = section_sizes_df.loc[(section_sizes_df['size'] >= lower_size) & (section_sizes_df['size'] <= upper_size)]
section_sizes_df.head()

,size,norm_sec_size,opt_sec_size
1956,1958,5,5
1957,1959,5,6
1958,1960,5,6
1959,1961,5,6
1960,1962,5,6


In [6]:
section_sizes_pivot_df = section_sizes_df.set_index('size').stack().reset_index(name='sec_size').rename(columns={'level_1':'sec_size_type'})
section_sizes_pivot_df.head()

,size,sec_size_type,sec_size
0,1958,norm_sec_size,5
1,1958,opt_sec_size,5
2,1959,norm_sec_size,5
3,1959,opt_sec_size,6
4,1960,norm_sec_size,5


In [7]:
file_name_arguments = [(number_from_text(input_file_name, size_regex),
                        number_from_text(input_file_name, sample_regex),
                        input_file_name)
                       for input_file_name in input_file_names]
file_name_arguments_df = pd.DataFrame(file_name_arguments, columns =['size', 'sample', 'path'])
file_name_arguments_df.head()

,size,sample,path
0,1958,1,SingMat_1958_1.txt
1,1958,2,SingMat_1958_2.txt
2,1973,1,SingMat_1973_1.txt
3,1973,2,SingMat_1973_2.txt
4,1988,1,SingMat_1988_1.txt


In [8]:
benchmark_tuples_df = pd.merge(section_sizes_pivot_df, file_name_arguments_df, on='size', how='inner')[['size', 'sec_size', 'sample', 'path']]
benchmark_tuples_df.head()

,size,sec_size,sample,path
0,1958,5,1,SingMat_1958_1.txt
1,1958,5,2,SingMat_1958_2.txt
2,1958,5,1,SingMat_1958_1.txt
3,1958,5,2,SingMat_1958_2.txt
4,1973,5,1,SingMat_1973_1.txt


In [9]:
benchmark_tuples_list = [tuple(benchmark_tuple) for benchmark_tuple in benchmark_tuples_df.values.tolist()]

benchmark_argument_tuples = [(synthesise_circuit,
                              size,
                              sec_size,
                              sample,
                              rf'..\{input_dir}\{input_file_name}')
                             for (size, sec_size, sample, input_file_name) in benchmark_tuples_list]

[print(argument_tuple) for argument_tuple in benchmark_argument_tuples[:min(10, len(benchmark_argument_tuples))]]

(<function synthesise_circuit at 0x000002740EA1BF70>, 1958, 5, 1, '..\\resources\\section_size_comparison\\input_matrices\\SingMat_1958_1.txt')
(<function synthesise_circuit at 0x000002740EA1BF70>, 1958, 5, 2, '..\\resources\\section_size_comparison\\input_matrices\\SingMat_1958_2.txt')
(<function synthesise_circuit at 0x000002740EA1BF70>, 1958, 5, 1, '..\\resources\\section_size_comparison\\input_matrices\\SingMat_1958_1.txt')
(<function synthesise_circuit at 0x000002740EA1BF70>, 1958, 5, 2, '..\\resources\\section_size_comparison\\input_matrices\\SingMat_1958_2.txt')
(<function synthesise_circuit at 0x000002740EA1BF70>, 1973, 5, 1, '..\\resources\\section_size_comparison\\input_matrices\\SingMat_1973_1.txt')
(<function synthesise_circuit at 0x000002740EA1BF70>, 1973, 5, 2, '..\\resources\\section_size_comparison\\input_matrices\\SingMat_1973_2.txt')
(<function synthesise_circuit at 0x000002740EA1BF70>, 1973, 6, 1, '..\\resources\\section_size_comparison\\input_matrices\\SingMat_1973_

[None, None, None, None, None, None, None, None, None, None]

In [ ]:
output_file_timestamp = datetime.now().strftime("%d%m%y_%H%M")

print(f'Benchmarking runtime performance in size range from {lower_size} to {upper_size} in {benchmark_processors} threads ...')

if __name__ ==  '__main__':
    with Pool(processes = benchmark_processors) as pool:
        result_tuples = pool.starmap(load_circuit_and_benchmark, benchmark_argument_tuples)

print(f"Saving results to '{output_dir} ...")

with open(rf'..\{output_dir}\{output_file_prefix}_{output_file_timestamp}.csv', mode='w', newline='') as output_file:
    writer = csv.DictWriter(output_file, fieldnames=output_file_fieldnames)
    writer.writeheader()

    for (size, run, sec_size, num_gates, process_time, initial_rss, final_rss) in result_tuples:

        writer.writerow({'size': size, 'run': run, 'sec_size': sec_size, 'num_gates': num_gates, 'process_time': process_time, 'initial_rss': initial_rss, 'final_rss': final_rss})

print('Done.')


Benchmarking runtime performance in size range from 1958 to 2048 in 10 threads ...
